In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.optimizers import SGD
#from sklearn.model_selection import cross_val_score
#from sklearn.model_selection import KFold
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
from pathlib import Path

# Helper functions
# from helper_functions import imgs_to_numpy

# Styles for miles
style.use('ggplot')

In [5]:
model_deepfake = keras.Sequential([
  keras.Input(shape=(256, 256, 3)),
#   layers.experimental.preprocessing.Normalization(variance=255*255),  FIXME - THis line
  layers.Conv2D(filters = 64, kernel_size = 4, strides = (1, 1), padding = 'same', activation = 'relu'),
  layers.MaxPooling2D((4, 4)),
  layers.Conv2D(filters = 32, kernel_size = 4, strides = (1, 1), padding = 'same', activation = 'relu'),
  layers.MaxPooling2D((4, 4)),
  layers.Conv2D(filters = 16, kernel_size = 4, strides = (1, 1), padding = 'same', activation = 'relu'),
  layers.MaxPooling2D((4, 4)),
  layers.Flatten(),
  layers.Dense(units = 4096, activation = 'relu'),
  layers.Dropout(rate = 0.1),
  layers.Dense(units = 256, activation = 'relu'), 
  layers.Dense(units = 1, activation = 'softmax')
])

In [ ]:
tf.keras.preprocessing.image_dataset_from_directory(
    directory,
    labels="inferred",
    label_mode="int",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(256, 256),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
)

In [3]:
model_deepfake.compile(optimizer = SGD(lr=1e-6), 
                       metrics = [keras.metrics.BinaryAccuracy(), 
                                  keras.metrics.Precision(), 
                                  keras.metrics.Recall()],
                       loss = keras.losses.BinaryCrossentropy())

In [4]:
model_deepfake.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 256, 256, 64)      3136      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 64, 64, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 64, 32)        32800     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 16)        8208      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 4, 16)          0         
_________________________________________________________________
flatten (Flatten)            (None, 256)               0

In [ ]:
# Save images as numpy array
for dsplit in ('train', 'valid', 'test'):
    imgs_to_numpy(dsplit=dsplit)

In [3]:
dsplit = 'train'

# Check that the split exists
if dsplit not in ('train', 'valid', 'test'):
    raise Exception('dsplit must be `train`, `test`, or `valid`')

# Load labeled dataframes
PATHDIR = Path('data')
DATADIR = Path('data/') / 'real_vs_fake' / 'real-vs-fake'
SAVEPATH = PATHDIR / 'data_array'

df = pd.read_csv(PATHDIR / f'{dsplit}.csv', header=0).drop(
    ['original_path', 'Unnamed: 0', 'label_str'], axis=1)

# Create containers for the image data
n = df.shape[0]
X = np.empty(shape=(n, 256, 256, 3))
y = df['label'].to_numpy()[np.newaxis].reshape(-1, 1)

In [5]:
# df['path'].tail()
img_path = 'train/fake/73ILM40K3Z.jpg'
img_idx = 99999
img = plt.imread(DATADIR / img_path)
X[img_idx, :, :, :] = img / 255.0

In [7]:
# X[img_idx, :, :, :]
with open(SAVEPATH / f'X_{dsplit}.npy', 'wb') as file:
    np.save(file, X)

In [8]:
with open(SAVEPATH / f'y_{dsplit}.npy', 'wb') as file:
        np.save(file, y)

In [ ]:
X = np.random.randn(n, 256, 256, 3)

In [ ]:
# Load in the training data
PATHTRAIN = Path('data') / 'data_array'
with tf.device('/device:GPU:0'):
    with open(PATHTRAIN / 'X_train.npy', 'rb') as f:
        X = np.load(f)
    
    with open(PATHTRAIN / 'y_train.npy', 'rb') as f:
        y = np.load(f)
print(X.shape)
print(y.shape)

In [ ]:
model_fit = model_deepfake.fit(X, 
                               y, 
                               epochs=20, 
                               batch_size=2000, 
                               validation_data=val_data)

In [ ]:
model_fit.history['binary_accuracy']

In [ ]:
plt.plot(model_fit.history['binary_accuracy'], 'g.-', label = 'Binary Accuracy')
plt.plot(model_fit.history['val_binary_accuracy'], 'r.-', label = 'Val Binary Accuracy')
plt.legend()
plt.title('Binary Accuracy over Time')
plt.show()

In [ ]:
plt.plot(model_fit.history['precision'], 'g.-', label = 'Precision')
plt.plot(model_fit.history['val_precision'], 'r.-', label = 'Val Precision')
plt.legend()
plt.title('Precision over Time')
plt.show()